In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from joblib import dump, load
import urllib
import os


def generate_token_sent(s):
    try:
        s = s.lower()
    except Exception as ex:
        s = str(each).lower()
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]
    sent = ""
    for i in range(len(tokens)):
        if i == 0:
            sent = sent + tokens[0]
        else:
            sent = sent + " " + tokens[i]
    return sent
    
def generate_token_sent_col(arr):
    new_arr = []
    for i in range(arr.size):
        new_arr.append(generate_token_sent(arr.iloc[i]))
    new_arr = np.array(new_arr)
    return new_arr

def generate_token_sent_col1(arr):
    new_arr = []
    for i in range(arr.size):
        new_arr.append(generate_token_sent(arr[i]))
    new_arr = np.array(new_arr)
    return new_arr

count_vect = load('vectroizer.pkl')
clf = load('svc.joblib')

In [2]:
def decode_back(s):
    try:
        s = urllib.parse.unquote(s)
        s = str.encode(s)
    except:
        return s
    return s.decode('utf8')

def predict(st):
    testpayload = np.array([decode_back(st)])
    testpayload_new = generate_token_sent_col1(testpayload)
    test_bow = count_vect.transform(testpayload_new)
    y_pred = clf.predict_proba(test_bow)
    print('Prediction for', st,' is ', y_pred[0][1])
    return y_pred[0][1]
    
def route(data):
    if isinstance(data, list):
        for each in data:
            try:
                s = each.lower()
            except Exception as ex:
                s = str(each).lower()
            if 'name' in s: 
                pload = s.split()[6][16:]
                ip=s.split()[0]
                score = predict(pload)
                if score >= 0.5:
                    os.system('iptables -t nat -A PREROUTING -p tcp --dport 80 -j DNAT --to-destination 192.168.1.114:80')
                    os.system('iptables -t nat -A POSTROUTING -p tcp -d 192.168.1.114 --dport 80 -j SNAT --to-source 192.168.1.113')
                    print(ip)
                else:
                    print('not XSS')
                #if score >= 0.8:
                    #os.system('cat /var/www/html/adm.php > /var/www/html/admin.php')
                #else:
                    #os.system('cat /var/www/html/admi.php > /var/www/html/admin.php')
    else:
        print('Route Function: Data type error')

In [3]:
fname = '/var/log/apache2/access.log'
while(True):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    if len(content) > 0:
        os.system('cat /var/log/apache2/access_reset.log > /var/log/apache2/access.log')
        route(content)

Prediction for rock  is  0.010440976693799653
not XSS
Prediction for %3cscript%3ealert(%22hello%22)%3c/script%3e  is  0.9999999999999699
192.168.1.103


KeyboardInterrupt: 